In [55]:
import os
import ee
import geopandas as gpd

from shapely.geometry  import box

ee.Authenticate()
ee.Initialize(project="ee-tiagogoncalves")

TILES_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/tiles"
FEATURE_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/embeddings"

os.makedirs(FEATURE_PATH, exist_ok=True)

In [ ]:
def get_download_url(xmin, ymin, xmax, ymax, year):
    geometry = ee.Geometry.Rectangle([[xmin, ymin], [xmax, ymax]], proj='EPSG:3857')

    s_date = ee.Date.fromYMD(year, 1, 1)
    e_date = s_date.advance(1, 'year')
                
    collection = ee.ImageCollection("GOOGLE/SATELLITE_EMBEDDING/V1/ANNUAL")
    
    image = collection.filterBounds(geometry).filterDate(s_date, e_date).first()
    
    return image.getDownloadURL({
        'scale': 10,
        'region': geometry,
        'format': 'GEO_TIFF'
    })


def download_from_url(filename, url):
    import requests
    response = requests.get(url)

    output_filename = f"{filename}-embeddings.tif"
    output_path = os.path.join(FEATURE_PATH, output_filename)
    
    if response.status_code == 200:
        with open(output_path, 'wb') as f:
            f.write(response.content)
        print(f"    ✓ Salvo: {filename}")
    else:
        print(f"    ❌ Erro ao baixar: Status {response.status_code}")

In [60]:
from time import sleep

filenames = [f for f in os.listdir(TILES_PATH) if os.path.isfile(os.path.join(TILES_PATH, f))]

for filename in filenames:
    gdf = gpd.read_file(os.path.join(TILES_PATH, filename))

    xmin, ymin, xmax, ymax = gdf.total_bounds
    print(xmin, ymin, xmax, ymax)

    for _ in range(2):
        try:
            url = get_download_url(xmin, ymin, xmax, ymax, 2024)

            download_from_url(filename.split('-')[0], url)

            break
        except Exception as e:
            print(f"    ❌ Erro ao baixar a imagem: {e}. Tentando novamente...")
            sleep(2)

    break
print("✅ Processamento concluído!")

saved_files = [f for f in os.listdir(FEATURE_PATH) if f.endswith('.tif')]
print(f"Total de arquivos GeoTIFF salvos: {len(saved_files)}")

3298060.0 9564778.0 3299080.0 9565798.0
    ❌ Erro ao baixar a imagem: Image is unbounded. Must specify a region or a pixel grid with dimensions.. Tentando novamente...
    ❌ Erro ao baixar a imagem: Image is unbounded. Must specify a region or a pixel grid with dimensions.. Tentando novamente...
✅ Processamento concluído!
Total de arquivos GeoTIFF salvos: 1


In [54]:
gdf = gpd.read_file('C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/tiles/1-mvp.gpkg')

gdf.total_bounds

array([3298060., 9564778., 3299080., 9565798.])